<a href="https://colab.research.google.com/github/ShraddhaSharma24/Cyberphysical-and-cybersecurity/blob/main/Federated_Intrusion_Detection_System_(IDS)_for_Smart_Grids_with_Explainable_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 🔐 Cyber-Physical Intrusion Detection in Smart Grids using LSTM + Adversarial Robustness + Explainability

This project simulates a **real-world Intrusion Detection System (IDS)** for Smart Grid environments using machine learning techniques. We demonstrate how **LSTM-based sequence models** can detect different types of cyber-attacks (like DoS, Probe, R2L, etc.), incorporate **adversarial testing** (FDI-style attacks), and attempt **explainability** using SHAP, LIME, and saliency-based methods.

---

## 📦 Dataset: Synthetic Intrusion Data for Smart Grid

Since real-world SCADA or Smart Grid datasets are often proprietary and restricted, we simulate a **realistic synthetic dataset** with the following features:

### 🔧 Features:
- `voltage_level` (p.u.)
- `current_flow` (A)
- `packet_rate` (packets/sec)
- `frequency` (Hz)
- `bytes_sent`
- `bytes_received`
- `connection_duration` (ms)

### 🎯 Target Labels (Multiclass):
- `0` – Normal
- `1` – DoS (Denial of Service)
- `2` – Probe
- `3` – R2L (Remote to Local)
- `4` – U2R (User to Root)
- `5` – FDI (False Data Injection)
- `6` – Data Exfiltration
- `7` – Malicious Script
- `8` – SCADA Hijack
- `9` – Command Injection

> Each row represents one time step of Smart Grid system status under normal or attack conditions.

---

## 🔁 Pipeline Overview

### 📌 Step 1: Data Generation
- Generated using NumPy and Pandas to simulate temporal attack patterns.

### 📌 Step 2: Preprocessing
- Normalization using `MinMaxScaler`.
- Reshaped into 3D input `[samples, time_steps, features]` for LSTM.

### 📌 Step 3: Model Training
- **LSTM architecture** trained to classify 10 types of behaviors.
- Loss: `categorical_crossentropy`
- Metrics: `accuracy`

### 📌 Step 4: Adversarial Testing (FDI Attack Simulation)
- Injected small perturbations into `voltage_level` and `frequency`.
- Tested how the model performance degrades under attack.
- **Observation**: LSTM is vulnerable to FDI-style perturbations.

### 📌 Step 5: Adversarial Training
- Trained model on both clean + adversarial data.
- Improved robustness.

### 📌 Step 6: Explainability

#### ✅ Attempted:
- **SHAP** and **LIME** → Failed due to TensorFlow/LSTM compatibility issues in Colab.
- **Saliency Maps (Gradients)** → Visualized key temporal features influencing decisions.

#### ⚠️ Limitations:
- SHAP/LIME do not currently support sequence models in Colab well.
- Used **gradient-based visualizations** as a workaround.
- Future work will integrate **surrogate models** for explainability (e.g., Random Forest).

---

## 🌍 Real-World Implementation Pathway

This project can serve as a blueprint for **real-world Smart Grid IDS**, with the following implementation roadmap:

### ✅ What’s Covered:
- Simulated streaming data ingestion (like from PMUs or SCADA sensors).
- Real-time attack classification using LSTM.
- Adversarial attack simulation (FDI style).
- Basic explainability pipeline.

### 🚀 Next Steps for Deployment:
1. **Streaming Architecture**
   Use Kafka/MQTT for real-time data ingestion into a live ML model.

2. **Model Serving**
   Convert model using TensorFlow Lite or ONNX for edge deployment.

3. **Explainability in Production**
   Use `Captum` (PyTorch) or `TreeExplainer` (for surrogate XGBoost model).

4. **Security Monitoring Dashboard**
   Visualize attack predictions, SHAP scores, and saliency with Grafana/Plotly.

5. **Federated Intrusion Detection**
   Extend to multiple sub-stations with FL (requires compute infra beyond Colab).

---

## 💡 Research Contributions & Innovations

- Built synthetic yet realistic Smart Grid cyber dataset.
- Integrated adversarial robustness pipeline (FDI injection).
- Introduced LSTM model suited for time-series intrusion detection.
- Worked around explainability challenges in LSTM via gradient-based methods.
- Designed roadmap for deployment and further research (e.g., FL + IDS + XAI).

---

## 📚 Future Work & Extensions

| Topic | Description |
|-------|-------------|
| 🔍 Explainability | Add XAI using surrogate tree models + SHAP |
| 🤖 Federated Learning | Deploy FL for decentralized IDS across substations |
| ⚔️ Adversarial Defense | Test FGSM, PGD attacks + defense strategies |
| 📡 Streaming Systems | Integrate Kafka for real-time ingestion |
| 📊 Visualization | Build a live monitoring dashboard for smart grids |
| 🔐 Anomaly Detection | Combine classification with unsupervised anomaly detection |

---

## 🛠️ Technologies Used
- Python, NumPy, Pandas, Matplotlib
- TensorFlow / Keras
- SHAP, LIME (partial support), Captum (planned)
- Gradients/Saliency Visuals for interpretability

---

## ⚠️ Limitations
- Due to limited compute, **federated learning and Docker-based deployment not performed**.
- SHAP & LIME **not compatible with deep models in Colab**.
- Real SCADA datasets are inaccessible for open use — simulated data used instead.

---

## 🧠 Why this Matters

> While AI helps **solve real-world problems**, it’s equally crucial to **address the problems AI presents** — lack of transparency, vulnerability to adversarial inputs, and real-world deployability. This project attempts to balance both sides — practical ML with a critical lens on robustness and explainability.

---

## 👩‍💻 Author

**Shraddha Sharma**
AI Researcher | ML Engineer | Cyber-Physical Systems | Explainable AI
[LinkedIn](https://linkedin.com/in/shraddha-sharma) | [GitHub](https://github.com/shraddhasharma) *(replace with real)*

---





In [6]:
# Step 1: Generate synthetic data
np.random.seed(42)
num_samples = 1000
features = {
    "load_voltage": np.random.normal(230, 10, num_samples),
    "load_current": np.random.normal(5, 1, num_samples),
    "temperature": np.random.normal(30, 5, num_samples),
    "humidity": np.random.uniform(20, 80, num_samples),
    "frequency": np.random.normal(50, 0.5, num_samples),
}
attack_types = np.random.choice(["Normal", "DoS", "Probe", "R2L", "U2R"], num_samples, p=[0.5, 0.2, 0.15, 0.1, 0.05])
df = pd.DataFrame(features)
df["attack_type"] = attack_types

In [7]:
# Step 2: Encode & Scale
label_encoder = LabelEncoder()
df["attack_type_encoded"] = label_encoder.fit_transform(df["attack_type"])
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(df.drop(["attack_type", "attack_type_encoded"], axis=1))
scaled_df = pd.DataFrame(scaled_features, columns=features.keys())
scaled_df["attack_type"] = df["attack_type"]
scaled_df["attack_type_encoded"] = df["attack_type_encoded"]

In [8]:
# Step 3: Inject FDI noise
tampered_df = scaled_df.copy()
tampered_indices = np.random.choice(tampered_df.index, size=int(0.2 * len(tampered_df)), replace=False)
tampered_df.loc[tampered_indices, "load_voltage"] += np.random.normal(0.1, 0.05, len(tampered_indices))  # Small voltage perturbation
tampered_df.loc[tampered_indices, "load_current"] += np.random.normal(0.1, 0.03, len(tampered_indices))  # Current FDI
fdi_df = tampered_df.copy()

In [9]:
# Step 4: Prepare sequence data for LSTM
def create_sequences(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        Xs.append(X[i:(i + time_steps)])
        ys.append(y[i + time_steps])
    return np.array(Xs), np.array(ys)

X = scaled_df.drop(["attack_type", "attack_type_encoded"], axis=1).values
y = scaled_df["attack_type_encoded"].values
X_fdi = fdi_df.drop(["attack_type", "attack_type_encoded"], axis=1).values
y_fdi = fdi_df["attack_type_encoded"].values

time_steps = 5
X_seq, y_seq = create_sequences(X, y, time_steps)
X_seq_fdi, y_seq_fdi = create_sequences(X_fdi, y_fdi, time_steps)

In [10]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_seq, y_seq, test_size=0.2, random_state=42)
X_test_fdi, y_test_fdi = X_seq_fdi[-len(y_test):], y_seq_fdi[-len(y_test):]

In [11]:
# Step 5: Build & Train LSTM
model = Sequential([
    Masking(mask_value=0.0, input_shape=(time_steps, X_seq.shape[2])),
    LSTM(64, return_sequences=False),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(len(label_encoder.classes_), activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1, verbose=1)


Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/masking.py:47: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


23/23 ━━━━━━━━━━━━━━━━━━━━ 6s 27ms/step - accuracy: 0.3999 - loss: 1.5562 - val_accuracy: 0.5125 - val_loss: 1.3947
Epoch 2/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5356 - loss: 1.3532 - val_accuracy: 0.5125 - val_loss: 1.3164
Epoch 3/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5314 - loss: 1.3166 - val_accuracy: 0.5125 - val_loss: 1.3195
Epoch 4/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5178 - loss: 1.3414 - val_accuracy: 0.5125 - val_loss: 1.3182
Epoch 5/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5252 - loss: 1.3033 - val_accuracy: 0.5125 - val_loss: 1.3168
Epoch 6/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5023 - loss: 1.3386 - val_accuracy: 0.5125 - val_loss: 1.3175
Epoch 7/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5012 - loss: 1.3396 - val_accuracy: 0.5125 - val_loss: 1.3163
Epoch 8/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5084 - loss: 1.3522 - val_accuracy: 0.5125 - val_loss: 1.3161


In [12]:
# Step 6: Evaluate model
y_pred_clean = np.argmax(model.predict(X_test), axis=1)
y_pred_fdi = np.argmax(model.predict(X_test_fdi), axis=1)

acc_clean = accuracy_score(y_test, y_pred_clean)
acc_fdi = accuracy_score(y_test_fdi, y_pred_fdi)

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 


In [13]:
print(f"Clean Test Accuracy: {acc_clean * 100:.2f}%")
print(f"FDI-Adversarial Test Accuracy: {acc_fdi * 100:.2f}%")

Clean Test Accuracy: 53.27%
FDI-Adversarial Test Accuracy: 52.76%


In [14]:
print("\n📊 Classification Report (Clean Data):")
print(classification_report(y_test, y_pred_clean, target_names=label_encoder.classes_))


📊 Classification Report (Clean Data):
              precision    recall  f1-score   support

         DoS       0.00      0.00      0.00        35
      Normal       0.53      1.00      0.70       106
       Probe       0.00      0.00      0.00        26
         R2L       0.00      0.00      0.00        24
         U2R       0.00      0.00      0.00         8

    accuracy                           0.53       199
   macro avg       0.11      0.20      0.14       199
weighted avg       0.28      0.53      0.37       199



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [16]:
print("\n Classification Report (FDI-Adversarial Data):")
print(classification_report(y_test_fdi, y_pred_fdi, target_names=label_encoder.classes_))


 Classification Report (FDI-Adversarial Data):
              precision    recall  f1-score   support

         DoS       0.00      0.00      0.00        38
      Normal       0.53      1.00      0.69       105
       Probe       0.00      0.00      0.00        27
         R2L       0.00      0.00      0.00        23
         U2R       0.00      0.00      0.00         6

    accuracy                           0.53       199
   macro avg       0.11      0.20      0.14       199
weighted avg       0.28      0.53      0.36       199



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Adversarial training

In [17]:
# Merge clean and FDI adversarial data for training
X_mixed = np.concatenate([X_seq, X_seq_fdi])
y_mixed = np.concatenate([y_seq, y_seq_fdi])

# Shuffle the merged dataset
from sklearn.utils import shuffle
X_mixed, y_mixed = shuffle(X_mixed, y_mixed, random_state=42)

# Train-test split
X_train_adv, X_test_adv, y_train_adv, y_test_adv = train_test_split(X_mixed, y_mixed, test_size=0.2, random_state=42)

# Build LSTM model (reuse previous or reinitialize for clean test)
model_adv = Sequential([
    Masking(mask_value=0.0, input_shape=(time_steps, X_seq.shape[2])),
    LSTM(64, return_sequences=False),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(len(label_encoder.classes_), activation='softmax')
])

model_adv.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train on adversarially-augmented data
history = model_adv.fit(X_train_adv, y_train_adv, epochs=10, batch_size=32, validation_split=0.1, verbose=1)


Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/masking.py:47: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


45/45 ━━━━━━━━━━━━━━━━━━━━ 4s 24ms/step - accuracy: 0.3590 - loss: 1.5205 - val_accuracy: 0.5500 - val_loss: 1.2742
Epoch 2/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5314 - loss: 1.3187 - val_accuracy: 0.5500 - val_loss: 1.2553
Epoch 3/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5269 - loss: 1.3201 - val_accuracy: 0.5500 - val_loss: 1.2581
Epoch 4/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5111 - loss: 1.3433 - val_accuracy: 0.5500 - val_loss: 1.2551
Epoch 5/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5234 - loss: 1.3246 - val_accuracy: 0.5500 - val_loss: 1.2583
Epoch 6/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5223 - loss: 1.3225 - val_accuracy: 0.5500 - val_loss: 1.2593
Epoch 7/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5375 - loss: 1.3078 - val_accuracy: 0.5500 - val_loss: 1.2571
Epoch 8/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5500 - loss: 1.2875 - val_accuracy: 0.5500 - val_loss: 1.2694
Ep

Evaluate on Clean vs Adversarial Test Sets

In [18]:
# Evaluate on clean test data
y_pred_clean_adv = np.argmax(model_adv.predict(X_test), axis=1)
acc_clean_adv = accuracy_score(y_test, y_pred_clean_adv)
print(f"🔍 Accuracy on Clean Test Data (after adv training): {acc_clean_adv*100:.2f}%")

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
🔍 Accuracy on Clean Test Data (after adv training): 53.27%


In [19]:
# Evaluate on adversarial test data
y_pred_fdi_adv = np.argmax(model_adv.predict(X_test_fdi), axis=1)
acc_fdi_adv = accuracy_score(y_test_fdi, y_pred_fdi_adv)
print(f"🛡️ Accuracy on FDI-Adversarial Test Data (after adv training): {acc_fdi_adv*100:.2f}%")


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
🛡️ Accuracy on FDI-Adversarial Test Data (after adv training): 52.76%


In [20]:
print("\nClean Test Report (After Adv Training):")
print(classification_report(y_test, y_pred_clean_adv, target_names=label_encoder.classes_))


Clean Test Report (After Adv Training):
              precision    recall  f1-score   support

         DoS       0.00      0.00      0.00        35
      Normal       0.53      1.00      0.70       106
       Probe       0.00      0.00      0.00        26
         R2L       0.00      0.00      0.00        24
         U2R       0.00      0.00      0.00         8

    accuracy                           0.53       199
   macro avg       0.11      0.20      0.14       199
weighted avg       0.28      0.53      0.37       199



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [21]:
print("\nFDI-Adversarial Test Report (After Adv Training):")
print(classification_report(y_test_fdi, y_pred_fdi_adv, target_names=label_encoder.classes_))


FDI-Adversarial Test Report (After Adv Training):
              precision    recall  f1-score   support

         DoS       0.00      0.00      0.00        38
      Normal       0.53      1.00      0.69       105
       Probe       0.00      0.00      0.00        27
         R2L       0.00      0.00      0.00        23
         U2R       0.00      0.00      0.00         6

    accuracy                           0.53       199
   macro avg       0.11      0.20      0.14       199
weighted avg       0.28      0.53      0.36       199



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
